In [3]:
#Allows relative imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
from src.models import *
from src.train_eval_helpers import *
from src.preprocessing import * 
import torch
import torch.optim
import torch.nn as nn
import torch.nn.functional as F

%load_ext autoreload
%autoreload 2 

#if module_path not in sys.path:
#sys.path.append(module_path)
print(module_path)
#print(sys.path)
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using : {}".format(device))
else:
    device = torch.device('cpu')
    print("Using : {}".format(device))
    
RANGE = range(12,17)
TRAINDIR = './TrainingData/'
files = os.listdir(TRAINDIR)
files.remove('readme.md')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\Users\richi\Documents\EPFL\Master\PDM\code
Using : cuda


### kfold & reset params

In [6]:
from sklearn.model_selection import KFold

In [33]:
import math
def xd(nb_epochs):
    return math.floor(nb_epochs/4)
    
a=xd(400)

In [35]:
a

100

In [9]:
X = torch.randn((1000,1,15,12))
kf = KFold(3)
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(i, train_index.shape, test_index.shape)


0 (666,) (334,)
1 (667,) (333,)
2 (667,) (333,)


In [36]:
kf = KFold(n_splits=4, shuffle=True)
X = torch.randn((1000,1,15,12))
y = torch.randn((1000,1))
print(X.shape)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index.shape, "TEST:", test_index[0:10])
    #X_train, X_test = X[train_index], X[test_index]
    #y_train, y_test = y[train_index], y[test_index]
X = torch.randn((1000,1,15,12), device='cuda')
train_data = X[train_index]
train_data.device, train_data.shape

torch.Size([1000, 1, 15, 12])
TRAIN: (750,) TEST: [ 0  3 19 26 28 29 30 31 32 33]
TRAIN: (750,) TEST: [ 1  8  9 10 12 13 17 20 21 22]
TRAIN: (750,) TEST: [ 2  4  7 16 25 36 38 40 42 43]
TRAIN: (750,) TEST: [ 5  6 11 14 15 18 23 24 39 47]


(device(type='cuda', index=0), torch.Size([750, 1, 15, 12]))

In [7]:
z = torch.tensor([[1,2,3,4,5,6],[1,2,3,4,6,7],[1,2,3,7,8,5]],dtype=torch.float32)
z.mean(0), z.var(0)

(tensor([1.0000, 2.0000, 3.0000, 5.0000, 6.3333, 6.0000]),
 tensor([0.0000, 0.0000, 0.0000, 3.0000, 2.3333, 1.0000]))

In [11]:
from src.models import deepcat_cnn
model = deepcat_cnn
mod1=model(12)
for x in list(mod1.children())[0:1]:
    if isinstance(x, nn.Conv2d) or isinstance(x,nn.Linear):
        print("before",x.weight[0])
        x.reset_parameters()
        print("after",x.weight[0])
        
mod13 = model(13)

print("before test", list(mod13.children())[0].weight[0][0][0:3])
mod13.reset_parameters()
print("after test", list(mod13.children())[0].weight[0][0][0:3])


In [4]:
#Reading data
train_normal = read_seq(TRAINDIR+files[0])
train_tumor = read_seq(TRAINDIR+files[2])
features, labels = generate_features_labels(train_tumor, train_normal, None)
model = deepcat_cnn(13)

Data device = cpu


In [39]:
logits,_,_ = model(data)
a = criterion(logits, targets)

In [40]:
a

tensor(0.5502, device='cuda:0', grad_fn=<NllLossBackward>)

In [46]:
#To Cuda
model.to(device)
data = features[13].detach().clone().to(device)
targets = labels[13].detach().clone().to(device)
#Options
criterion = nn.CrossEntropyLoss()
lr = 0.00125
optimizer = optim.Adam(model.parameters(),lr)
mini_batch_size = 500
nb_epochs = 300
kfold = 5
#run
train_loss, val_loss, acc, AUC = kfold_cv(model, criterion, optimizer, nb_epochs, lr, 
                                          kfold, mini_batch_size, data, targets,
                                          verbose=True)

Crossvalidating : fold =  0
Current stats at epoch = 0 :
Train loss = 0.7151500634525133
 Val loss = 0.6582126617431641
 Acc = 0.6252089602139752
AUC = 0.6578864469829644
Current stats at epoch = 100 :
Train loss = 0.5458535487237184
 Val loss = 0.4560486972332001
 Acc = 0.7803410230692076
AUC = 0.8637353444081723
Current stats at epoch = 200 :
Train loss = 0.5306760655797046
 Val loss = 0.4476974606513977
 Acc = 0.7843530591775326
AUC = 0.8665227182258676
Crossvalidating : fold =  1
Current stats at epoch = 0 :
Train loss = 0.7054688075314397
 Val loss = 0.6617169380187988
 Acc = 0.6232029421598128
AUC = 0.6224599436477565
Current stats at epoch = 100 :
Train loss = 0.5619006778882898
 Val loss = 0.4734615683555603
 Acc = 0.7793380140421263
AUC = 0.8638338683525614
Current stats at epoch = 200 :
Train loss = 0.5376173542893451
 Val loss = 0.449660986661911
 Acc = 0.7907054496823804
AUC = 0.8678586614706721
Crossvalidating : fold =  2
Current stats at epoch = 0 :
Train loss = 0.7066477

In [51]:
import pandas as pd
auc_df = pd.DataFrame(AUC)

In [54]:
auc_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.657886,0.724961,0.760515,0.784362,0.794971,0.808602,0.812132,0.817898,0.823067,0.827765,...,0.858950,0.860181,0.861427,0.864081,0.863669,0.861815,0.860045,0.863413,0.859660,0.861575
1,0.622460,0.683882,0.692142,0.729648,0.754925,0.770167,0.776574,0.786772,0.792960,0.801620,...,0.875011,0.874138,0.874340,0.873552,0.870188,0.873370,0.873104,0.871926,0.871687,0.872723
2,0.721975,0.750553,0.764927,0.747385,0.777075,0.778750,0.792266,0.793957,0.798259,0.809575,...,0.876927,0.879573,0.878711,0.878703,0.875748,0.876086,0.879857,0.881815,0.878180,0.877792
3,0.672091,0.718630,0.756479,0.776390,0.788340,0.794415,0.799202,0.802753,0.805936,0.807331,...,0.872302,0.872143,0.870535,0.871908,0.870627,0.874095,0.871017,0.871326,0.869837,0.868634
4,0.719824,0.744019,0.766776,0.775211,0.786552,0.795530,0.801092,0.805470,0.811255,0.814434,...,0.858460,0.861179,0.861991,0.859870,0.859502,0.862465,0.861737,0.861108,0.861492,0.861904
